In [ ]:
# Installing Dependencies
!pip install crewai crewai_tools gradio pandas

In [ ]:
# Environment Configuration
from google.colab import userdata
import os
import crewai
gemini_api_key = userdata.get('Gemini_api_key')
os.environ["GEMINI_API_KEY"] = gemini_api_key

In [ ]:
# Tools Initialization
from crewai_tools import FileReadTool, DirectoryReadTool
import pandas as pd

file_read_tool = FileReadTool()
directory_read_tool = DirectoryReadTool()

In [ ]:
from crewai import LLM
llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)

In [ ]:
# Defining the Agents
from crewai import Agent, Task, Crew

# Resume Analyzer Agent
resume_analyzer = Agent(
    role='Resume Analyzer',
    goal='Extract and analyze key information from resumes including skills, experience, education, and achievements',
    backstory='You are an expert HR analyst skilled at parsing resumes and identifying relevant candidate information.',
    tools=[file_read_tool, directory_read_tool],
    llm=llm,
    verbose=True
)

# Job Requirements Parser Agent
job_parser = Agent(
    role='Job Requirements Parser',
    goal='Parse job descriptions to identify required skills, experience levels, and qualifications',
    backstory='You are a recruitment specialist who understands job requirements and can identify key selection criteria.',
    llm=llm,
    verbose=True
)

# Candidate Scorer Agent
candidate_scorer = Agent(
    role='Candidate Scorer',
    goal='Score candidates based on their match with job requirements using comprehensive evaluation criteria',
    backstory='You are a talent acquisition expert who can objectively evaluate candidate suitability for specific roles.',
    llm=llm,
    verbose=True
)

# Ranking and Report Generator Agent
report_generator = Agent(
    role='Recruitment Report Generator',
    goal='Rank candidates and generate comprehensive recruitment reports with detailed justifications',
    backstory='You are a senior recruiter who creates detailed candidate assessments and hiring recommendations.',
    llm=llm,
    verbose=True
)

In [ ]:
# Task Setup and Execution
# Task 1: Resume Analysis
task1 = Task(
    description='Analyze all provided resumes and extract key information including: skills, work experience, education, certifications, and notable achievements. Return structured data for each candidate.',
    expected_output='Structured analysis of all resumes with extracted key information.',
    agent=resume_analyzer
)

# Task 2: Job Requirements Parsing
task2 = Task(
    description="Parse the job description for '{job_title}' and identify: required skills, experience level, educational requirements, preferred qualifications, and key responsibilities.",
    expected_output='Structured job requirements with clear criteria for candidate evaluation.',
    agent=job_parser
)

# Task 3: Candidate Scoring
task3 = Task(
    description=(
        'Score each candidate against the job requirements using the following criteria:\n'
        '1. Skills Match (40%): How well candidate\'s skills align with required skills\n'
        '2. Experience Relevance (30%): Quality and relevance of work experience\n'
        '3. Education Fit (15%): Educational background alignment\n'
        '4. Additional Qualifications (15%): Certifications, projects, achievements\n\n'
        'Provide a score (0-100) for each candidate with detailed justification.'
    ),
    expected_output='Scored candidates with detailed breakdown of evaluation criteria.',
    agent=candidate_scorer
)

# Task 4: Ranking and Report Generation
task4 = Task(
    description=(
        'Generate a comprehensive recruitment report including:\n'
        '- Ranked list of top candidates\n'
        '- Detailed analysis for each candidate\n'
        '- Strengths and areas of concern\n'
        '- Hiring recommendations\n'
        '- Summary of candidate pool quality\n\n'
        'Format the report in markdown with clear sections and professional presentation.'
    ),
    expected_output='Complete recruitment report with candidate rankings and detailed assessments.',
    agent=report_generator
)

crew = Crew(
    agents=[resume_analyzer, job_parser, candidate_scorer, report_generator],
    tasks=[task1, task2, task3, task4],
    verbose=True
)

result = crew.kickoff(inputs={
    'job_title': 'Senior Software Engineer',
    'resumes_path': '/path/to/resumes/'
})

In [ ]:
# Optional UI with Gradio
import gradio as gr

def process_recruitment(job_description, resume_files):
    result = crew.kickoff(inputs={
        'job_description': job_description,
        'resumes': resume_files
    })
    return result

with gr.Blocks() as demo:
    gr.Markdown('# AI Recruiter Assistant')

    with gr.Row():
        with gr.Column():
            job_desc = gr.Textbox(label='Job Description', lines=10)
            resume_files = gr.Files(label='Upload Resumes')
            submit_btn = gr.Button('Analyze Candidates')

        with gr.Column():
            output = gr.Markdown(label='Recruitment Report')

    submit_btn.click(
        process_recruitment,
        inputs=[job_desc, resume_files],
        outputs=output
    )

 demo.launch()